In [2]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
# import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.rc('font', family='SimHei', size=13)
%matplotlib inline

import seaborn as sns
sns.set()

# import plotly.graph_objs as go
# import plotly.plotly as py
# import plotly.offline as of
# of.offline.init_notebook_mode(connected=True)

In [3]:
app_launch_log = pd.read_csv(
    '../data/B/app_launch_log.txt', delimiter='\t', header=None)
user_register_log = pd.read_csv(
    '../data/B/user_register_log.txt', delimiter='\t', header=None)
video_create_log = pd.read_csv(
    '../data/B/video_create_log.txt', delimiter='\t', header=None)
user_activity_log = pd.read_csv(
    '../data/B/user_activity_log.txt', delimiter='\t', header=None)

print(app_launch_log.shape, user_register_log.shape, video_create_log.shape,
      user_activity_log.shape)

(252496, 2) (51480, 4) (35849, 2) (21072403, 6)


In [4]:
user_register_log.columns=['user_id','register_day','register_type','device_type']
app_launch_log.columns=['user_id','day']
video_create_log.columns=['user_id','day']
user_activity_log.columns=['user_id','day','page','video_id','author_id','action_type']

## 数据划分

In [5]:
user=user_register_log[user_register_log.register_day.values<17].user_id.unique()

user_video=video_create_log[video_create_log.day<17]
user_reg=user_register_log[user_register_log.register_day<17]
user_lau=app_launch_log[app_launch_log.day<17]
user_act=user_activity_log[user_activity_log.day<17]

u1=video_create_log[video_create_log.day >= 17].user_id.unique()
u2=user_register_log[user_register_log.register_day >= 17].user_id.unique()
u3=app_launch_log[app_launch_log.day >= 17].user_id.unique()
u4=user_activity_log[user_activity_log.day >= 17].user_id.unique()
u=np.unique(np.concatenate([u1,u2,u3,u4]))

#特征1 操作次数
group = user_act.groupby(user_act.user_id)
act_count = group.size()

#特征2 video创建次数
group = user_video.groupby(user_video.user_id)
video_count = group.size()

#特征3 注册时间（注册日期）
reg_day = user_reg.iloc[:, :2]
reg_day = reg_day.set_index(reg_day.user_id).loc[:, 'register_day']

#特征4 启动次数
group = user_lau.groupby(user_lau.user_id)
lau_count = group.size()

#特征5 最后一次使用日期
group = user_act.groupby(user_act.user_id)
act_maxday = group.day.max()

#特征6 最后一次创建video
group = user_video.groupby(user_video.user_id)
video_maxday = group.day.max()

#特征7 最大操作次数
group = user_act.groupby(['user_id', 'day'])
act_max = group.size()
v = [act_max[i].max() for i in act_max.index.levels[0]]
act_max = pd.Series(v, index=act_max.index.levels[0])

#特征8 最大连续使用天数
from itertools import groupby

def count_series_day(days):
    lst = days.tolist()
    fun = lambda x: x[1] - x[0]
    return np.max(
        [len([v for i, v in g]) for k, g in groupby(enumerate(lst), fun)])

group = user_act.groupby(user_act.user_id)
series_days = group.day.apply(lambda df: count_series_day(df.values))

#特征9 连续使用次数
def count_series_day(days):
    lst = days.tolist()
    fun = lambda x: x[1] - x[0]
    return len([[v for i, v in g] for k, g in groupby(enumerate(lst), fun)])

group = user_act.groupby(user_act.user_id)
series_days_count = group.day.apply(lambda df: count_series_day(df.values))

#特征10 最大连续几天没有使用





data = pd.concat(
    [act_count, video_count, reg_day, lau_count, act_maxday, video_maxday,act_max,series_days,series_days_count],
    axis=1)
data.columns = [
    'act_count', 'video_count', 'reg_day', 'lau_count', 'act_maxday',
    'video_maxday','act_max','series_days','series_days_count'
]
data = data.fillna(0)
label = [1 if i in u else 0 for i in data.index]
data['target'] = label

data['use_days']=17-data.reg_day.values

data['mean_act']=data.act_count.values/data.use_days.values

data['mean_video']=data.video_count.values/data.use_days.values

data['mean_series']=data.series_days.values/data.use_days.values

data['mean_lau']=data.lau_count.values/data.use_days.values

In [22]:
data.head()

,act_count,video_count,reg_day,lau_count,act_maxday,video_maxday,act_max,series_days,series_days_count,target,use_days,mean_act,mean_video,mean_series,mean_lau
user_id,,,,,,,,,,,,,,,
8,96.0,0.0,9,4,13.0,0.0,39.0,2.0,87.0,1,8,12.000000,0.0,0.250000,0.500000
73,36.0,0.0,2,3,5.0,0.0,31.0,2.0,32.0,0,15,2.400000,0.0,0.133333,0.200000
129,1.0,0.0,12,1,12.0,0.0,1.0,1.0,1.0,1,5,0.200000,0.0,0.200000,0.200000
228,22.0,0.0,13,1,13.0,0.0,22.0,1.0,22.0,1,4,5.500000,0.0,0.250000,0.250000
351,176.0,0.0,3,1,3.0,0.0,176.0,1.0,176.0,0,14,12.571429,0.0,0.071429,0.071429


In [23]:
X=data.loc[:,['act_maxday','video_maxday','act_max','mean_act','mean_video','mean_series','mean_lau']]
y=data.target.values

In [25]:
X=np.array(X)

In [28]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size = 0.3)

In [32]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

bagging_clf=BaggingClassifier(DecisionTreeClassifier(max_leaf_nodes=10,random_state=666),
                    n_estimators=10000,max_samples=5000,
                    bootstrap=True,oob_score=True,
                    n_jobs=-1,
                    max_features=6,bootstrap_features=True)
bagging_clf.fit(X_train,y_train)
# bagging_clf.oob_score_
bagging_clf.score(X_test,y_test)

0.781524313780131

In [34]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf=KNeighborsClassifier()
knn_clf.fit(X_train,y_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [35]:
knn_clf.score(X_test,y_test)

0.7480841577260694

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X)

In [ ]:
user=user_register_log[user_register_log.register_day.values<17].user_id.unique()

user_video=video_create_log[video_create_log.day<17]
user_reg=user_register_log[user_register_log.register_day<17]
user_lau=app_launch_log[app_launch_log.day<17]
user_act=user_activity_log[user_activity_log.day<17]

u1=video_create_log[video_create_log.day >= 17].user_id.unique()
u2=user_register_log[user_register_log.register_day >= 17].user_id.unique()
u3=app_launch_log[app_launch_log.day >= 17].user_id.unique()
u4=user_activity_log[user_activity_log.day >= 17].user_id.unique()
u=np.unique(np.concatenate([u1,u2,u3,u4]))

#特征1 操作次数
group = user_act.groupby(user_act.user_id)
act_count = group.size()

#特征2 video创建次数
group = user_video.groupby(user_video.user_id)
video_count = group.size()

#特征3 注册时间（注册日期）
reg_day = user_reg.iloc[:, :2]
reg_day = reg_day.set_index(reg_day.user_id).loc[:, 'register_day']

#特征4 启动次数
group = user_lau.groupby(user_lau.user_id)
lau_count = group.size()

#特征5 最后一次使用日期
group = user_act.groupby(user_act.user_id)
act_maxday = group.day.max()

#特征6 最后一次创建video
group = user_video.groupby(user_video.user_id)
video_maxday = group.day.max()

#特征7 最大操作次数
group = user_act.groupby(['user_id', 'day'])
act_max = group.size()
v = [act_max[i].max() for i in act_max.index.levels[0]]
act_max = pd.Series(v, index=act_max.index.levels[0])

#特征8 最大连续使用天数
from itertools import groupby

def count_series_day(days):
    lst = days.tolist()
    fun = lambda x: x[1] - x[0]
    return np.max(
        [len([v for i, v in g]) for k, g in groupby(enumerate(lst), fun)])

group = user_act.groupby(user_act.user_id)
series_days = group.day.apply(lambda df: count_series_day(df.values))

#特征9 连续使用次数
def count_series_day(days):
    lst = days.tolist()
    fun = lambda x: x[1] - x[0]
    return len([[v for i, v in g] for k, g in groupby(enumerate(lst), fun)])

group = user_act.groupby(user_act.user_id)
series_days_count = group.day.apply(lambda df: count_series_day(df.values))

#特征10 最大连续几天没有使用





data = pd.concat(
    [act_count, video_count, reg_day, lau_count, act_maxday, video_maxday,act_max,series_days,series_days_count],
    axis=1)
data.columns = [
    'act_count', 'video_count', 'reg_day', 'lau_count', 'act_maxday',
    'video_maxday','act_max','series_days','series_days_count'
]
data = data.fillna(0)
label = [1 if i in u else 0 for i in data.index]
data['target'] = label

data['use_days']=17-data.reg_day.values

data['mean_act']=data.act_count.values/data.use_days.values

data['mean_video']=data.video_count.values/data.use_days.values

data['mean_series']=data.series_days.values/data.use_days.values

data['mean_lau']=data.lau_count.values/data.use_days.values